# 빅 콘데스트
2020 9 28

*팀명:긍정의씨앗*


팀원 : 이재원, 박세진, 최우진, 최호진, 황인범
챔피언스리그 참가

# **중분류/소분류 생성**


* 원데이터의 '상품군'을 좀 더 세세하게 집단으로 쪼개서 분석 할 수 있는 방법이 없을까?


## 1. NS SHOP+ 홈페이지 크롤링



* 원데이터 상품명에 대한 중분류와 소분류 카테고리 정보를 CSV파일로 가져오기


* NS SHOP+ 홈페이지 주소 : https://www.nsmall.com/

### 크롤링을 위한 패키지 불러오기

In [ ]:
from urllib.parse import quote_plus
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait   
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException   
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import random

#  Chrome webdriver 를 읽는다.
wd = webdriver.Chrome('C:\\Users\\hwang in beom\\Desktop\\chromedriver.exe')

### 웹드라이버로 페이지 이동하며 크롤링

In [ ]:
count = 0


# 해당 페이지에서  script를 사용해 페이지 버튼을 클릭하면서 소분류의 데이터를 크롤링 한 후 다음 페이지로 이동하는 부분
def crawling_goobne(url, main_category, middle_category, subclass_category ,wd, count):
    wd.get(url)

    RESULT_DIRECTORY = '../'
    results = []
    list_name = []
    if len(last) == 0:
        print('none')
    else:
        for i in range(1, int(last[0])+1):
            script = 'movePage(%d)' % i
            wd.execute_script(script)  # js 실행
            time.sleep(2 + random.uniform(0.01, 0.1))              

            html = wd.page_source
            bs = BeautifulSoup(html, 'html.parser')

            tags = bs.findAll('span', attrs={'class': 'inform pr10'})
            print(tags)
            for tag in tags:
                list_name.append(tag.get_text())
        count += 1
        table = pd.DataFrame(list_name)
        print(table)
        table.to_csv('./data/{}-{}-{}.csv'.format(main_category, middle_category, subclass_category), encoding='euc-kr', mode='w', errors=None,)
        time.sleep(5 + random.uniform(0.01, 0.1)) 

### BeautifulSoup 이용해 소분류 url 가져오기


In [ ]:
# nsmall 해당 페이지로 이동
url_str ='http://www.nsmall.com/CategoryDisplay?searchType=search&storeId=13001&langId=-9&catalogId=97001&categoryId=1583591&cate1Code=1583596&cate2Code=1583567&cate3Code=1583591&menuUri=NSSubCategoryPageLayoutView&selfId='
req = requests.get(url_str)

# HTML 소스를 BeautifulSoup를 통해 가져오기
html = req.text
bs = BeautifulSoup(html, 'html.parser')
url_list = bs.find('li', attrs={'class': 'cate_List third'}).find('ul', attrs={'class': 'depth1'}).find_all("li")


# 해당 카테고리의 중분류 url 가져오기 
url_list_subclass = []
for i in range(0,len(url_list)):
    start = (str(url_list[i]).find('href=')) + 6
    end = str(url_list[i]).find('"><span>')
    url_list[i] = ("http://www.nsmall.com/" + str(url_list[i])[start:end]).replace('amp;', '')

    req = requests.get(url_list[i])  
    html = req.text
    bs = BeautifulSoup(html, 'html.parser')
    url_list_subclass.append(bs.find('dl', attrs={'class': 'lst hnl'}).find_all("li"))

# 해당 중분류의 소분류 카테고리의 url 가져오기
subclass_url_list = []
for i in url_list_subclass:
    count = 0
    for j in i:
        if count == 0:
            count += 1
            continue
        start = (str(j).find('<a href=')) + 9
        j = (str(j)[int(start):])
        end = str(j).find('" id')
        subclass_url_list.append(("http://www.nsmall.com/" + str(j)[:end]).replace('amp;', ''))


### 소분류 url을 통해 상품명에 따른 카테고리 정보 가져오기

In [ ]:
# 해당 소분류의 url을 통해 각 상품의 상품군/중분류/소분류를 가져오는 부분
for i in subclass_url_list:
    if count < 6:
        count += 1
        continue
    url_str = i
    print(url_str)
    req = requests.get(url_str)
    ## HTML 소스 가져오기
    html = req.text
    # print(html)
    bs = BeautifulSoup(html, 'html.parser')
    pagging = bs.find('div', attrs={'class': 'lst_paging'})
    last = re.findall("\d+", str(pagging.find('a', attrs={'class': 'last'})))
    main_category = bs.find('li', attrs={'class': 'cate_List second'}).find('a', attrs={'href': '#;'}).get_text()
    middle_category = bs.find('li', attrs={'class': 'cate_List third'}).find('a', attrs={'href': '#;'}).get_text()
    subclass_category = bs.find('div', attrs={'class': 'soCate'}).find('strong').get_text()+str(count)

    main_category = main_category.replace('/', ',')
    middle_category = middle_category.replace('/', ',')
    subclass_category = subclass_category.replace('/', ',')
    crawling_goobne(url_str, main_category, middle_category, subclass_category, wd, count)
    count += 1

## 2. 상품에 대한 해당 카테고리 가져오기


* 상품명의 특정 단어를 검색하여, NS SHOP+ 홈페이지에서 정의된 중분류, 소분류 카테고리 정보 가져오는 알고리즘 생성

* ex) "그릴" 검색  => 상품군 : 주방 / 중분류 : 주방가전 / 소분류 : 조리,가열기기

In [ ]:
import pandas as pd
import numpy as np


# 해당 중분류의 csv 파일을 가져오는 작업

while(1):
    find_class = input("찾으실 분류 이름을 입력하세요 :")
    if find_class == '0':
        break
    df = pd.read_csv("data/" + find_class +"-소분류.csv")
    coulmn = df.columns

    # 소분류로 분류 된 값들을 이름으로 검색 하는 작업
    while(1):
        find_name = input("찾을 이름을 입력하세요 :")
        if find_name == '0':
            break
        result = False
        for i in coulmn:
            if find_class not in i:
                continue
            np_data = np.array(df[i].tolist())
            list_data = list(np_data)
            for j in list_data:
                if find_name in j:
                    result = True
                    break
            if result == True:
                print("******")
                print(i)
                print("******")
                result = False

print("고생하셨습니다")